In [ ]:
import pandas as pd

In [ ]:
data_path='/content/drive/MyDrive/BIGDATA_STUDY/Final_project_ZIP/파이널_코드정리본/'
data = pd.read_csv(data_path+'waterpark_최종테스트용.csv', encoding= 'CP949')

In [ ]:
data_path='/content/drive/MyDrive/BIGDATA_STUDY/Final_project_ZIP/파이널_코드정리본/'
data = pd.read_csv(data_path+'waterpark_ALL_REVIEW.csv')

# 1.맞춤법 교정

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-yojzautg
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-yojzautg
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=4998fbb4cb67214fb1190029b2810dfb700af674afec6480060df78a092ee669
  Stored in directory: /tmp/pip-ephem-wheel-cache-ocr3vl4y/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker
from json.decoder import JSONDecodeError
from time import sleep
from xml.etree.ElementTree import ParseError
from tqdm.auto import tqdm

In [ ]:
data_pre = pd.DataFrame(columns = ['rating', 'review', 'name'])

In [ ]:
#Hanspell 맞춤법 교정
for i in tqdm(range(len(data))):
    data_sample = data[i:i + 1]
    try:
        data_sample['review'] = data_sample['review'].apply(lambda x : spell_checker.check(str(x)).checked)
        data_pre = pd.concat([data_pre, data_sample], axis = 0)

    except JSONDecodeError as e: 
        print(f'에러 JSONDecode')
        continue
    except ParseError as e:
        print(f'에러, Parse')
        continue

  0%|          | 0/193269 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
data=data_pre

#  2.Kiwi_Word2Vector Clustering

In [ ]:
!pip3 install gensim==3.8.3
!pip install vocab
!pip install kiwipiepy

In [ ]:
from pandas import DataFrame  as df
import numpy as np
import re
import pickle
from gensim import corpora
from wordcloud import WordCloud
from kiwipiepy import Kiwi
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import os

## 2-1 토크나이징 및 명사 추출

In [ ]:
Data_list=data['review'].tolist()

In [ ]:
#키위 토크나이저로 토큰 분류
kiwi = Kiwi()
data_word=[]

for i in tqdm(range(len(Data_list))):
    try:
        data_word.append(kiwi.tokenize(Data_list[i]))
    except Exception as e:
        continue

In [ ]:
#리스트를 요소별로 명사만 추출한 후 리스트로 저장
morph_list = []
for i in tqdm(range(len(data_word))):
  nouns=[]
  for j in range(len(data_word[i])):
    if data_word[i][j][1] in ['NNG','NNP']:
      #NNG:일반명사, NNP:고유명사, VV:동사, VA:형용사, VX:보조용언, VCP:긍정지시사(이다), VCN:부정지시사(아니다), XR:어근
      nouns.append(data_word[i][j][0])
  morph_list.append(nouns)
morph_list

## 2-2 클러스터링-토픽 및 토픽의 핵심키워드 확인용

In [ ]:
# Word2Vec embedding
from gensim.models import Word2Vec
model = Word2Vec(morph_list, size=100, window = 5, min_count=5, workers=4, iter=100, sg=1) #size 벡터 크기, window 앞뒤로 몇개 단어 연관, workers cpu코어사용수, sg=1 skipgram사용
#기존 모델의 파라미터 model = Word2Vec(morph_list, size=100, window = 5, min_count=50, workers=4, iter=100, sg=1)

In [ ]:
#적정 클러스터링 수 확인
from sklearn.cluster import KMeans
embedding_model=model
word_vectors = embedding_model.wv.syn0 # 어휘의 feature vector
num_clusters = int(word_vectors.shape[0]/50) # 어휘 크기의 1/5나 평균 5단어
print(num_clusters)

In [ ]:
# num_clusters = 8 #클러스터링 임의 조정
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

idx = list(idx)
names = embedding_model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

In [ ]:
import csv
num=0
for c in range(num_clusters):
    # 클러스터 번호를 출력
    print("\ncluster {}".format(c))
    
    words = []
    cluster_values = list(word_centroid_map.values())
    for i in range(len(cluster_values)):
      if (cluster_values[i] == c):
          words.append(list(word_centroid_map.keys())[i])            
    
    if len(words)>10:
      print(words)
      
      with open(data_path+'topic_data/'+"topic("+str(num)+")_.csv", 'w') as file:
        writer = csv.writer(file)
        writer.writerow(words)
      num+=1

## 2-3 WORD2VECTOR 토픽키워드 추출

In [ ]:
# check embedding result
sim_words=[]
topic_list=[['구매','할인','예매','결제','티켓'],['시설','샤워', '목욕탕','파도', '기구','슬라이드','튜브'],
['음식','식당', '맛', '푸드', '밥'],['초등','유치원','아이','가족','성인'],['서비스','안전','위험','직원','기분']]

#기존 토픽
#[['구매','할인','예매','결제','티켓'],['시설','샤워', '목욕탕','파도', '수풀', '기구', '주차','셔틀','슬라이드'],
# ['음식','식당', '맛', '푸드', '밥'],['초등','유치원','아이','가족','성인'],['서비스','안전','위험','직원','기분']]

for i in range(len(topic_list)):
  check_list=[]
  for topic in topic_list[i]: 
    word_list = model.wv.most_similar(topic, topn=50)
    for word in word_list:
      if len(word[0])>1 and word[1]>0.3:
        check_list.append(word)
    
  check_list=set(check_list)
  sim_words.append(check_list)
  print(topic_list[i][0],len(check_list),check_list)

In [ ]:
#구매,위생,시설,음식,동반,서비스 토픽별 딕셔너리로 저장
#동일 토픽내 중복 발생 시 높은 점수로 입력

#내림차순 정렬
for i in range(len(sim_words)):
  sim_words[i]=list(sim_words[i])
  sim_words[i].sort()

#딕셔너리 저장
buy_list={}
for i in range(len(sim_words[0])):
  buy_list[sim_words[0][i][0]]=sim_words[0][i][1]

install_list={}
for i in range(len(sim_words[1])):
  install_list[sim_words[1][i][0]]=sim_words[1][i][1]
  
food_list={}
for i in range(len(sim_words[2])):
  food_list[sim_words[2][i][0]]=sim_words[2][i][1]
  
visit_list={}
for i in range(len(sim_words[3])):
  visit_list[sim_words[3][i][0]]=sim_words[3][i][1]
  
service_list={}
for i in range(len(sim_words[4])):
  service_list[sim_words[4][i][0]]=sim_words[4][i][1]

In [ ]:
#중복 제거 (점수 기준)
topic_list = [buy_list,install_list,food_list,visit_list,service_list]
for topic1 in topic_list:
  for key in topic1:
    for topic2 in topic_list:
      if key in topic2 and topic1[key]>topic2[key]:
        del(topic2[key])
keyword_dic={}

keyword_dic['buy']=list(buy_list.keys())
keyword_dic['food']=list(food_list.keys())
keyword_dic['installation']=list(install_list.keys())
keyword_dic['service']=list(service_list.keys())
keyword_dic['visit_type']=list(visit_list.keys())
topic_dict=keyword_dic

In [ ]:
#토픽-키워드 저장
with open(data_path+'topic_dict0610.pkl','wb') as f:
  pickle.dump(keyword_dic, f)

# 3.리뷰 평가 

In [ ]:
data

In [ ]:
# 연결어미, 종결어미 기준 토큰 분리
def sep_token(tokens):
  result = []
  tmp = []
  for token in tokens:
    tmp.append(tuple((token)))
    if token.tag in ['EC', 'EF']:
      result.append(tmp)
      tmp = []
    elif token == tokens[-1]:
      result.append(tmp)
  return result

# 명사가 포함되지 않을 경우 이전 문장과 연결
def change_sep_token(token_lists):
  tmp_lists = []
  tmp = []
  for i in range(len(token_lists)):
    tmp += token_lists[i]
    if not i == len(token_lists) - 1:
      if ('NNG' in list(zip(*token_lists[i + 1]))[1] or 'NNP' in list(zip(*token_lists[i + 1]))[1] or 'NNB' in list(zip(*token_lists[i + 1]))[1]):
        tmp_lists.append(tmp)
        tmp = []
    else:
        tmp_lists.append(tmp)
  return tmp_lists

In [ ]:
data['token'] = data['review'].apply(lambda x : kiwi.tokenize(str(x)))
data['sep_token'] = data['token'].apply(lambda x : sep_token(x))
data['change_sep_token'] = data['sep_token'].apply(lambda x : change_sep_token(x))

In [ ]:
new_list=[]
for topic in topic_dict:
    for i in range(len(topic)):
        new_list.append(kiwi.tokenize(topic_dict[topic][i]))

## Bert model transformer 이용

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")

In [ ]:
import numpy as np
for topic in ['buy','food', 'installation', 'service', 'visit_type']:
  data[topic] = 0
  for i in range(len(data)):
    data[topic][i] = np.array([0, 0, 0], dtype = object)
data.drop(columns = ['token', 'sep_token'], inplace = True)
data.rename(columns = {'change_sep_token':'token'}, inplace = True)

In [ ]:
# 문장 내 일반,고유 명사 추출
def extract_topic(sentence):
  for tok, tag, start, len in sentence:
    if tag in ['NNP', 'NNG']:
      for topic in topic_dict.keys():
        if tok in topic_dict[topic]:
          return topic


def stat_review(sentence, review):
  start = sentence[0][2]
  end = sentence[-1][2] + sentence[-1][3]
  stat = classifier(review[start:end])[0]
  return stat


def eval_review(token, review):
  result = []
  review = review
  for sentence in token:
    topic = extract_topic(sentence)
    stat = stat_review(sentence, review)
    result.append(tuple((topic, stat)))
  return result

In [ ]:
cnt = len(data)
for i in tqdm(range(cnt), desc = '진행도', mininterval = 0.01):
  for topic, stat in eval_review(data['token'][i], data['review'][i]):
    if topic != None:
      if stat['label'] == 'positive':
        if stat['score'] >= 0.9:
          data[topic][i][0] += 1
          data[topic][i][1] += 1
      else:
        if stat['score'] >= 0.9:
          data[topic][i][0] -= 1
          data[topic][i][2] += 1


In [ ]:
data.to_csv(data_path+'통합코드_테스트_결과.csv', index = False)
data